In [1]:
suppressMessages(library(survminer))
suppressMessages(library(survival))
suppressMessages(library(forestplot))
suppressMessages(library(ggsci))
suppressMessages(library(dplyr))
suppressMessages(library(adjustedCurves))
suppressMessages(library(tidyverse))
suppressMessages(library(cmprsk))
suppressMessages(library(caret))
library(tidyr)

source("funcs/rfuncs.R")

## 1. Load Dataset
---

In [2]:
data.df <- read.table("data/processed/AL_with_ccp_03.tsv", sep="\t", header=T, row.names='Code.ID')

# Rename columns
column_mapping <- c("LC.Isotype"="Kappa_or_lambda_PCD", 
                    "BU.Cardiac.Stage"="BU__BNP_based__cardiac_staging",
                    "BMPC.%"="Bone_marrow_plasma_cells____"
                   )

data.df <- data.df %>% rename(!!!column_mapping)

# Filter for time from admission >=0
data.df <- data.df[data.df$time_from_admission>=0,]

# Collapse Race
data.df <- data.df %>%
   mutate(Race = ifelse(Race %in% c('American_Indian_Alaska_Native','Multiracial','Native_Hawaiian_Pacific', 'Unknown/other'), 'Other', Race))

# Fix NULL values
data.df <- data.df %>% mutate(BU.Cardiac.Stage = ifelse(BU.Cardiac.Stage=="", NA, BU.Cardiac.Stage))
data.df <- data.df %>% mutate(Primary_organ = ifelse(Primary_organ=="", NA, Primary_organ))
data.df <- data.df %>% mutate(LC.Isotype = ifelse(LC.Isotype=="", NA, LC.Isotype))

# assign cluster to var
data.df$m02q3_cluster_n <- factor(data.df$m02q3_cluster_n, levels=c("Low","Intermediate","High"))
data.df <- data.df[complete.cases(data.df[,"m02q3_cluster_n"]),]
data.df$m02q4_cluster_n <- factor(data.df$m02q4_cluster_n, levels=c("Low","Intermediate-Low","Intermediate","High"))
data.df$fna3_cluster_n <- factor(data.df$fna3_cluster_n, levels=c("Low","Intermediate","High"))
data.df$BU.Cardiac.Stage <- as.factor(data.df$BU.Cardiac.Stage)

# As Factor
data.df$LC.Isotype <- as.factor(data.df$LC.Isotype)
data.df$Sex <- as.factor(data.df$Sex)
data.df$Race <- as.factor(data.df$Race)
data.df$Era <- as.factor(data.df$Era)

In [3]:
# Load ESKD Data
eskd.df <- read.table("data/processed/treatment_eskd.tsv", sep="\t", header=T, row.names="Code.ID")
eskd.df <- eskd.df[rownames(data.df),]
eskd.df <- cbind(data.df, eskd.df[,c("RRT.Type","treatment_eskd","time_to_eskd","time_to_eskd_from_admission","CR_event","CR_time")])
eskd.df <- eskd.df[complete.cases(eskd.df$time_to_eskd_from_admission),]

# Filter for whether or not patients had ESKD at baseline
eskd.df <- eskd.df[eskd.df$time_to_eskd_from_admission>0,]

# Set factor orders
eskd.df$Renal.Stage..Palladini. <- factor(eskd.df$Renal.Stage..Palladini., levels=c("Stage I","Stage II","Stage III"))
eskd.df$fna3_cluster_n <- factor(eskd.df$fna3_cluster_n, levels=c("High","Low","Intermediate"))


In [4]:
table(eskd.df[,c("fna3_cluster_n","Renal.Stage..Palladini.")])

              Renal.Stage..Palladini.
fna3_cluster_n Stage I Stage II Stage III
  High             218       67         1
  Low              339      228        50
  Intermediate     125      350       146

In [5]:
table(eskd.df[,c("fna3_cluster_n","treatment_eskd")])

              treatment_eskd
fna3_cluster_n   0   1
  High         354   8
  Low          638  88
  Intermediate 533 135

In [6]:
pdf("figures/cluster_fna3_km_curve_renal_stageI.pdf", h=8, w=6)

fit <- survfit(Surv(time_to_eskd, treatment_eskd) ~ fna3_cluster_n, 
data=eskd.df[eskd.df$Renal.Stage..Palladini.=="Stage I",])
plotSurv(fit, eskd.df, "Subgroup", NULL, palette=c("#ca4a29","#62b67d","#69a9c8"), 
         surv.median.line="v", xlim=c(0,12), ylim=c(0.75,1), break.time.by=2)
dev.off()

Warning message:
"Removed 5 rows containing missing values (`geom_step()`)."
Warning message:
"Removed 2 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 1 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 2 rows containing missing values (`geom_segment()`)."
Warning message:
"Removed 5 rows containing missing values (`geom_step()`)."
Warning message:
"Removed 2 rows containing missing values (`geom_point()`)."
Warning message:
"Removed 1 rows containing missing values (`geom_text()`)."
Warning message:
"Removed 2 rows containing missing values (`geom_segment()`)."


pdf 
  2

## 2. Cox Regression Modeling
---

In [7]:
# mod.1 <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n, data=eskd.df)
# mod.2 <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini., data=eskd.df)
# mod.3 <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini., data=eskd.df)
# mod.4 <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + eGFR + log2(X24_hr_UTP+1), data=eskd.df)
# mod.5 <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + eGFR + log2(X24_hr_UTP+1) + Albumin, data=eskd.df)

In [8]:
# Baseline Model
eskd.crude <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n, data=eskd.df)
eskd.stage <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini., data=eskd.df)
eskd.baseline <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Age + Sex + Race + LC.Isotype, data=eskd.df)
cox.zph(eskd.baseline)

           chisq df     p
Age         4.09  1 0.043
Sex         0.23  1 0.631
Race        3.30  3 0.347
LC.Isotype  4.13  1 0.042
GLOBAL     11.84  6 0.066

In [22]:
eskd.crude$concordance

concordant   discordant       tied.x       tied.y      tied.xy  concordance 
8.736900e+04 2.778700e+04 7.745800e+04 2.000000e+00 7.000000e+00 6.546668e-01 
         std 
1.663896e-02

In [9]:
# Baseline models
eskd.baseline.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), data=eskd.df, tt=function(x,t,...) x * log(t))
eskd.baseline.cluster.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))
eskd.baseline.stage.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))
eskd.baseline.cluster.stage.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))


# Labs baseline for Schoenfiled test
eskd.labs <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Age + Sex + Race + eGFR + log2(X24_hr_UTP+1) + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE)
cox.zph(eskd.labs)

                      chisq df       p
Age                   3.753  1  0.0527
Sex                   0.347  1  0.5557
Race                  2.087  3  0.5546
eGFR                 17.850  1 2.4e-05
log2(X24_hr_UTP + 1)  0.161  1  0.6882
GLOBAL               22.060  7  0.0025

In [13]:
# Revisions
#getCoef(coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n*Era + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t)))


In [19]:
coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n, data=eskd.df, x=TRUE)$concordance

concordant   discordant       tied.x       tied.y      tied.xy  concordance 
8.736900e+04 2.778700e+04 7.745800e+04 2.000000e+00 7.000000e+00 6.546668e-01 
         std 
1.663896e-02

In [18]:
coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini., data=eskd.df, x=TRUE)$concordance

concordant   discordant       tied.x       tied.y      tied.xy  concordance 
9.797300e+04 1.332500e+04 4.866300e+04 2.000000e+00 6.000000e+00 7.645895e-01 
         std 
1.318695e-02

In [113]:
# For use with adjusted curves
eskd.baseline.cluster <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + Sex + Race + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE)

In [106]:
# Proteinuria & eGFR
eskd.labs.cluster.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP+1) + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))
eskd.labs.stage.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP+1) + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))
eskd.labs.cluster.stage.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP+1) + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))

# Proteinuria & eGFR & Albumin
eskd.labs.cluster.v2.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP+1) + Albumin + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))
eskd.labs.stage.v2.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP+1) + Albumin + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))
eskd.labs.cluster.stage.v2.t <- coxph(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP+1) + Albumin + strata(LC.Isotype) + strata(Era), data=eskd.df, x=TRUE, tt=function(x,t,...) x * log(t))


In [107]:
cox_models <- list(eskd.crude, 
    eskd.baseline.cluster.t, eskd.labs.cluster.t, eskd.labs.cluster.v2.t,
    eskd.baseline.cluster.stage.t, eskd.labs.cluster.stage.t, eskd.labs.cluster.stage.v2.t
)

cox_summary <- coxSummary(cox_models, var="fna3_cluster_n")
cox_summary$High <- "Reference"

# # Add events
result <- eskd.df %>% 
    filter(!is.na(eval(fna3_cluster_n))) %>% 
    group_by(fna3_cluster_n) %>% 
    summarize(Events = sum(treatment_eskd)) %>%
    column_to_rownames(var = "fna3_cluster_n")

cox_summary <- rbind(t(result), cox_summary)
cox_summary

,High,Low,Intermediate
,<chr>,<chr>,<chr>
Events,8,88,135
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n",Reference,3.2 (1.55-6.61),6.5 (3.18-13.3)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era)",Reference,3.25 (1.57-6.75),6.65 (3.24-13.6)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP + 1) + strata(LC.Isotype) + strata(Era)",Reference,3.44 (1.37-8.61),6.44 (2.56-16.2)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP + 1) + Albumin + strata(LC.Isotype) + strata(Era)",Reference,4.18 (1.3-13.5),4.57 (1.35-15.4)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era)",Reference,2.11 (0.903-4.95),2.32 (0.987-5.46)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP + 1) + strata(LC.Isotype) + strata(Era)",Reference,2.67 (1.06-6.74),4.16 (1.62-10.7)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP + 1) + Albumin + strata(LC.Isotype) + strata(Era)",Reference,3.54 (1.09-11.5),3.7 (1.09-12.6)


In [108]:
cox_models <- list(eskd.stage, eskd.baseline.stage.t, eskd.labs.stage.t, eskd.labs.stage.v2.t,
    eskd.baseline.cluster.stage.t, eskd.labs.cluster.stage.t, eskd.labs.cluster.stage.v2.t
)

cox_summary <- coxSummary(cox_models, var="Renal.Stage..Palladini.")
cox_summary$Stage.I <- "Reference"

# # Add events
result <- eskd.df %>% 
    filter(!is.na(eval(Renal.Stage..Palladini.))) %>% 
    group_by(Renal.Stage..Palladini.) %>% 
    summarize(Events = sum(treatment_eskd)) %>%
    column_to_rownames(var = "Renal.Stage..Palladini.")
row.names(result) <- c("Stage.I","Stage.II","Stage.III")

cox_summary <- rbind(t(result), cox_summary)
cox_summary

,Stage.I,Stage.II,Stage.III
,<chr>,<chr>,<chr>
Events,23,127,70
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini.",Reference,5.93 (3.8-9.24),14.4 (8.98-23.1)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era)",Reference,5.91 (3.79-9.24),14.7 (9.04-23.8)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP + 1) + strata(LC.Isotype) + strata(Era)",Reference,4.02 (2.42-6.69),6.52 (3.2-13.3)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP + 1) + Albumin + strata(LC.Isotype) + strata(Era)",Reference,2.69 (1.55-4.68),3.69 (1.67-8.16)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + strata(Era)",Reference,5.06 (3.18-8.07),12.1 (7.2-20.2)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP + 1) + strata(LC.Isotype) + strata(Era)",Reference,3 (1.77-5.08),4.04 (1.9-8.57)
"Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n + Renal.Stage..Palladini. + Age + tt(Age) + Sex + Race + eGFR + tt(eGFR) + log2(X24_hr_UTP + 1) + Albumin + strata(LC.Isotype) + strata(Era)",Reference,2.43 (1.4-4.23),3.27 (1.47-7.27)


In [109]:
pdf("figures/cluster_fna3_km_eskd_curve.pdf", h=8, w=6)
fit <- survfit(Surv(time_to_eskd_from_admission, treatment_eskd) ~ fna3_cluster_n, data=eskd.df)
plotSurv(fit, eskd.df, "Subgroup", NULL, palette=c("#ca4a29","#62b67d","#69a9c8"), 
         surv.median.line="v", break.time.by=2)
dev.off()

pdf 
  2

## 3. Adjusted Cumulative Incidence of ESKD
---

In [115]:
summary(eskd.baseline.cluster)

Call:
coxph(formula = Surv(time_to_eskd_from_admission, treatment_eskd) ~ 
    fna3_cluster_n + Age + Sex + Race + strata(LC.Isotype) + 
        strata(Era), data = eskd.df, x = TRUE)

  n= 1756, number of events= 231 
   (6 observations deleted due to missingness)

                                coef exp(coef)  se(coef)      z Pr(>|z|)    
fna3_cluster_nLow           1.170997  3.225208  0.372031  3.148  0.00165 ** 
fna3_cluster_nIntermediate  1.892574  6.636429  0.366535  5.163 2.42e-07 ***
Age                        -0.006494  0.993527  0.006331 -1.026  0.30500    
Sexmale                     0.264477  1.302749  0.139279  1.899  0.05758 .  
RaceBlack                  -0.201554  0.817459  0.550963 -0.366  0.71450    
RaceOther                   0.738659  2.093126  0.841569  0.878  0.38010    
RaceWhite                   0.286793  1.332149  0.457891  0.626  0.53109    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

                           exp(coef) exp(-coef) l

In [114]:
# Adjusted survival curve without bootstrap
eskd.adjsurv <- adjustedsurv(data=eskd.df,
                        variable="fna3_cluster_n",
                        ev_time="time_to_eskd_from_admission",
                        event="treatment_eskd",
                        method="direct",
                        outcome_model=eskd.baseline.cluster,
                        conf_int=T,
                        n_cores=8,
                        n_boot=1000,
                        bootstrap=TRUE
                       )

In [116]:
saveRDS(eskd.adjsurv, "data/analysis/eskd_adjsurv.rds")

In [23]:
#eskd.adjsurv <- readRDS("data/analysis/eskd_adjsurv.rds")

In [117]:
eskd.adjsurv$ate_object

     Average treatment effect 

 - Treatment            : fna3_cluster_n (3 levels: "High" "Low" "Intermediate")
 - Event                : treatment_eskd (cause: 1, censoring: 0)
 - Time  [min;max]      : time_to_eskd_from_admission [0.00274;23.9]
 - Eval. time                     :     0 0.0054757015742642 0.0109514031485284
              <char>         <char> <int>              <int>              <int>
      number at risk         High     362                360                358
      number at risk          Low     726                724                724
      number at risk Intermediate     668                668                665
 0.0191649555099247 0.0246406570841889 0.0355920602327173 0.0383299110198494
              <int>              <int>              <int>              <int>
                356                355                351                351
                723                720                720                720
                662                660      

In [120]:
pdf("figures/adjusted_eskd_clust.pdf")
plot(eskd.adjsurv, conf_int=F, custom_colors=c("#ca4a29","#62b67d","#69a9c8"), cif=T) + xlim(0,12.5) + ylim(0,0.45)
dev.off()

Warning message:
"Removed 48 rows containing missing values (`geom_step()`)."


pdf 
  2

In [124]:
source("funcs/rfuncs.R")

In [125]:
getIncidenceGroup(eskd.adjsurv)

,1,5,10
,<chr>,<chr>,<chr>
High,1.28 (0.495-2.26),3.72 (1.48-6.43),6.72 (2.75-11.3)
Low,4.16 (3.07-5.39),11.7 (9.16-14.5),20.4 (16.1-24.5)
Intermediate,8.4 (6.37-10.6),22.7 (18.9-27),37.6 (31.8-43.8)


In [126]:
getRiskDiff(eskd.adjsurv, c("Low vs. High","Intermediate vs. High","Intermediate vs. Low"))

group,1,5,10
<fct>,<chr>,<chr>,<chr>
Low vs. High,2.82 (1.46-4.19),7.96 (4.46-11.5),13.5 (7.47-19.5)
Intermediate vs. High,6.95 (4.68-9.21),18.8 (14-23.7),30.4 (23-37.7)
Intermediate vs. Low,4.12 (1.77-6.48),10.9 (6.04-15.7),16.9 (9.54-24.2)


In [127]:
# P-values
adj_test <- adjusted_curve_test(eskd.adjsurv, from=0, to=12)
summary(adj_test)

------------------------------------------------------------------
   Test of the Difference between two adjusted Survival Curves
------------------------------------------------------------------

Using the interval: 0 to 12 

                         ABC ABC SE 95% CI (lower) 95% CI (upper) P-Value
High vs. Low          1.0802 0.2361         0.6178         1.5188       0
High vs. Intermediate 2.4855 0.2919         1.9391         3.0619       0
Low vs. Intermediate  1.4053 0.2801         0.8766         1.9493       0
                      N Boot
High vs. Low             901
High vs. Intermediate    901
Low vs. Intermediate     901
------------------------------------------------------------------


In [128]:
clust.adjci <- eskd.adjsurv
clust.adjci$adjsurv$surv <- 1-clust.adjci$adjsurv$surv

In [129]:
pdf("figures/adjusted_eskd_clustdiff_low_high.pdf")
plot_curve_diff(clust.adjci, group_1="Low", group_2="High", conf_int=F, color="blue") + ylim(0,0.4) + xlim(0,12.5)
dev.off()

Warning message:
"Removed 1 row containing missing values (`geom_step()`)."


pdf 
  2

In [130]:
pdf("figures/adjusted_eskd_clustdiff_intermediate_high.pdf")
plot_curve_diff(clust.adjci, group_1="Intermediate", group_2="High", conf_int=F, color="red")+ ylim(0,0.4) + xlim(0,12.5)
dev.off()

Warning message:
"Removed 1 row containing missing values (`geom_step()`)."


pdf 
  2

In [131]:
pdf("figures/adjusted_eskd_clustdiff_high_intermediate.pdf")
plot_curve_diff(clust.adjci, group_1="Intermediate", group_2="Low", conf_int=F, color="green")+ ylim(0,0.4) + xlim(0,12.5)
dev.off()

Warning message:
"Removed 1 row containing missing values (`geom_step()`)."


pdf 
  2

## 4. Fine Gray Model
---

In [132]:
eskd.baseline.cluster.t

Call:
coxph(formula = Surv(time_to_eskd_from_admission, treatment_eskd) ~ 
    fna3_cluster_n + Age + tt(Age) + Sex + Race + strata(LC.Isotype) + 
        strata(Era), data = eskd.df, x = TRUE, tt = function(x, 
    t, ...) x * log(t))

                                coef exp(coef)  se(coef)      z        p
fna3_cluster_nLow           1.179837  3.253843  0.372258  3.169  0.00153
fna3_cluster_nIntermediate  1.895019  6.652673  0.366602  5.169 2.35e-07
Age                        -0.004992  0.995020  0.006640 -0.752  0.45217
tt(Age)                    -0.002851  0.997153  0.003712 -0.768  0.44244
Sexmale                     0.262840  1.300619  0.139275  1.887  0.05913
RaceBlack                  -0.189598  0.827292  0.551239 -0.344  0.73088
RaceOther                   0.758421  2.134903  0.842105  0.901  0.36779
RaceWhite                   0.290840  1.337551  0.457936  0.635  0.52536

Likelihood ratio test=65.59  on 8 df, p=3.694e-11
n= 1756, number of events= 231 
   (6 observations dele

In [133]:
# Fit with CSC
# adjustedcif (X)

# then plot - make sure y-axis is adjusted cumulative incidence function


In [134]:
CR <- cuminc(eskd.df$CR_time, eskd.df$CR_event, group=eskd.df$fna3_cluster_n, cencode=0)

p <- ggcompetingrisks(CR, xlabs="Time from Diagnosis (Yr)", 
    ylab="Cumulative Incidence", multiple_panels = F, palette=c("#ca4a29","#69a9c8","#62b67d"), conf.int = T)
p$mapping <- aes(x = time, y = est, colour = group, linetype = event)

pdf("figures/competingrisks_ci.pdf")
p + labs(linetype = "event", colour = "group") + geom_line(size = 1) + xlim(0,12.5) + scale_fill_manual(values = c("lightgrey","lightgrey"))
dev.off()


6 cases omitted due to missing values


Warning message:
"Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
i Please use `linewidth` instead."
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Warning message:
"Removed 152 rows containing missing values (`geom_line()`)."
Warning message:
"Removed 152 rows containing missing values (`geom_line()`)."


pdf 
  2

In [10]:
# Addd log UTP
eskd.df$log2_UTP <- log2(eskd.df$X24_hr_UTP+1)

fg.crude <- runFineGray(eskd.df, c("fna3_cluster_n"))
fg.baseline <- runFineGray(eskd.df, c("fna3_cluster_n", "Age", "Sex", "Race", "LC.Isotype"))
fg.renal <- runFineGray(eskd.df, c("fna3_cluster_n", "Renal.Stage..Palladini.", "Age", "Sex", "Race", "LC.Isotype"))
fg.labs <- runFineGray(eskd.df, c("fna3_cluster_n", "Age", "Sex", "Race", "LC.Isotype", "log2_UTP", "eGFR"))
fg.renal_labs <- runFineGray(eskd.df, c("fna3_cluster_n", "Renal.Stage..Palladini.", "Age", "Sex", "Race", "LC.Isotype", "eGFR"))
fg.bu <- runFineGray(eskd.df, c("fna3_cluster_n", "BU.Cardiac.Stage", "Age", "Sex", "Race", "LC.Isotype"))
fg.all <- runFineGray(eskd.df, c("fna3_cluster_n", "BU.Cardiac.Stage", "Renal.Stage..Palladini.", "Age", "Sex", "Race", "LC.Isotype"))

6 cases omitted due to missing values
6 cases omitted due to missing values
6 cases omitted due to missing values
6 cases omitted due to missing values
6 cases omitted due to missing values
6 cases omitted due to missing values
293 cases omitted due to missing values
293 cases omitted due to missing values
173 cases omitted due to missing values
173 cases omitted due to missing values
896 cases omitted due to missing values
896 cases omitted due to missing values
896 cases omitted due to missing values
896 cases omitted due to missing values


In [17]:
head(eskd.df)

,Date_of_admission_to_center,Date_of_birth,Date_of_diagnosis,Date_of_death,Date_of_last_visit,Cardiopulmonary_symptom_onset,Renal_signs_symptom_onset,Neurologic_symptom_onset,Amyloid_type,Age,...,fna3_cluster_n,m02q4_cluster_n,m02q3_cluster_n,RRT.Type,treatment_eskd,time_to_eskd,time_to_eskd_from_admission,CR_event,CR_time,log2_UTP
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,...,<fct>,<fct>,<fct>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
6102,1996-12-16,1944-03-20,1996-11-08,2017-01-28,2012-08-27,,1996-05-15,,AL,52,...,Low,Low,Low,,0,20.2217659,20.117727584,2,20.117727584,11.028597
9338,1994-07-11,1933-12-21,1993-06-24,1997-08-02,1997-03-24,,1993-03-15,,AL,59,...,Low,Intermediate-Low,Intermediate,Peritoneal Dialysis,1,1.4757016,0.429842574,1,0.429842574,0.000000
9419,1994-07-11,1920-02-06,1994-04-15,1994-08-21,1994-07-11,,1994-01-15,,AL,74,...,Intermediate,Intermediate,Intermediate,,0,0.3504449,0.112251882,2,0.112251882,14.090361
9421,1994-07-18,1937-04-26,1994-05-04,1995-02-24,1994-07-18,1993-04-15,,,AL,57,...,High,High,High,,0,0.8104038,0.605065024,2,0.605065024,6.658211
9422,1994-07-25,1948-05-05,1994-05-06,2018-04-10,2001-07-23,1992-12-15,,1994-03-15,AL,46,...,High,High,High,KidneyTransplant,1,21.7878166,21.568788501,1,21.568788501,8.005625
9423,1994-07-25,1936-02-10,1994-06-17,1994-12-01,1994-07-25,,1994-05-15,,AL,58,...,Intermediate,Intermediate,Intermediate,Peritoneal Dialysis,1,0.1095140,0.005475702,1,0.005475702,12.200592


In [11]:
getCoef((fg.crude$eskd.crr))

,coef,exp.coef.,se.coef.,z,p.value,coef_exp,ci_lower,ci_upper,HR
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
fna3_cluster_n.Low,1.755887,5.788579,0.3660703,4.796584,1.6e-06,5.79,2.82,11.9,5.79 (2.82-11.9)
fna3_cluster_n.Intermediate,2.315191,10.126858,0.3609510,6.414142,1.4e-10,10.10,4.99,20.5,10.1 (4.99-20.5)


In [198]:
df <- rbind(fg.crude$coef, fg.baseline$coef, fg.labs$coef, fg.renal_labs$coef)
df <- df[df$covariate %in% c("fna3_cluster_n.Low","fna3_cluster_n.Intermediate"),]
df <- df[df$Risk=="ESKD",c("covariate","HR","formula")]
df <- data.frame(pivot_wider(df, names_from = covariate, values_from = HR))
rownames(df) <- df$formula
df$formula <- NULL

# Add events
result <- eskd.df %>% 
    filter(!is.na(eval(fna3_cluster_n))) %>% 
    group_by(fna3_cluster_n) %>% 
    summarize(Events = sum(CR_event)) %>%
    column_to_rownames(var = "fna3_cluster_n")


colnames(df) <- c("Low","Intermediate")
df$High <- "Reference"

rbind(t(result),df)



,High,Low,Intermediate
,<chr>,<chr>,<chr>
Events,602,948,953
~ fna3_cluster_n,Reference,5.79 (2.82-11.9),10.1 (4.99-20.5)
~ fna3_cluster_n + Age + Sex + Race + LC.Isotype,Reference,5.86 (2.86-12),10.1 (4.99-20.5)
~ fna3_cluster_n + Age + Sex + Race + LC.Isotype + log2_UTP + eGFR,Reference,6.97 (2.87-16.9),9.88 (4.02-24.3)
~ fna3_cluster_n + Renal.Stage..Palladini. + Age + Sex + Race + LC.Isotype + eGFR,Reference,4.79 (2.12-10.8),5.44 (2.34-12.7)
